# 1. Introducition
The following code contains the creation and training of an RNN Model using Keras and TensorFlow. The project was originally started in Kaggle but was moved to Jupyter Notebook so as to utilize hardware acceleration. The aim of the model is to decide whether tweets entered are one of eight emotions. The emotions are as followed: {'joy', 'surprise', 'sadness', 'anger', 'trust', 'fear', 'disgust', 'anticipation'}. Many of the porcesses taken and utilized were thanks to the GPU-Accelerated performance of the Jupyter Notebook. With that I was able to perform more epochs as well as create more versions as well.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 2 Importing Files
Due to the JSON being a Nested Json, the original df did not fully contain the information needed. Due to this another dataframe was created with json_normalize so as to have the tweet_id, and the text, which were very importan. The tweet_id was necessary so as to add combine the other dataframes that contained the training and test data. The reason for this was so as to have the dataframe be able to share the tweet text via the tweet_id. Afterwards, separate dataframes where created to have the Training and Testing Data Separate from each other, as well as the emotion labels.

In [2]:
#Creating Twitter DF From Json Files
twitterDM = "Twitter/tweets_DM.json"
twitter_df = pd.read_json(twitterDM, lines=True)
twitter_df.head()

_score          _index                                            _source  \
0     391  hashtag_tweets  {'tweet': {'hashtags': ['Snapchat'], 'tweet_id...   
1     433  hashtag_tweets  {'tweet': {'hashtags': ['freepress', 'TrumpLeg...   
2     232  hashtag_tweets  {'tweet': {'hashtags': ['bibleverse'], 'tweet_...   
3     376  hashtag_tweets  {'tweet': {'hashtags': [], 'tweet_id': '0x1cd5...   
4     989  hashtag_tweets  {'tweet': {'hashtags': [], 'tweet_id': '0x2de2...   

            _crawldate   _type  
0  2015-05-23 11:42:47  tweets  
1  2016-01-28 04:52:09  tweets  
2  2017-12-25 04:39:20  tweets  
3  2016-01-24 23:53:05  tweets  
4  2016-01-08 17:18:59  tweets

In [3]:
#Getting the Tweet Inform From the Source Column of the Original DF
twitterDM_df = pd.json_normalize(twitter_df._source, record_prefix=None)
twitterDM_df.columns = twitterDM_df.columns.str.replace('^tweet.', '')
twitterDM_df

C:\Users\galve\AppData\Local\Temp\ipykernel_4328\3164084898.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  twitterDM_df.columns = twitterDM_df.columns.str.replace('^tweet.', '')


hashtags  tweet_id  \
0                             [Snapchat]  0x376b20   
1          [freepress, TrumpLegacy, CNN]  0x2d5350   
2                           [bibleverse]  0x28b412   
3                                     []  0x1cd5b0   
4                                     []  0x2de201   
...                                  ...       ...   
1867530  [mixedfeeling, butimTHATperson]  0x316b80   
1867531                               []  0x29d0cb   
1867532                               []  0x2a6a4f   
1867533                               []  0x24faed   
1867534                    [Sundayvibes]  0x34be8c   

                                                      text  
0        People who post "add me on #Snapchat" must be ...  
1        @brianklaas As we see, Trump is dangerous to #...  
2        Confident of your obedience, I write to you, k...  
3                      Now ISSA is stalking Tasha 😂😂😂 <LH>  
4        "Trust is not the same as faith. A friend is s...  
...                                                    ...  
1867530  When you buy the last 2 tickets remaining for ...  
1867531  I swear all this hard work gone pay off one da...  
1867532  @Parcel2Go no card left when I wasn't in so I ...  
1867533  Ah, corporate life, where you can date <LH> us...  
1867534             Blessed to be living #Sundayvibes <LH>  

[1867535 rows x 3 columns]

In [4]:
#Creating Data_Identification DF
twitterID = "Twitter/data_identification.csv"
twitid_df = pd.read_csv(twitterID)
twitid_df

tweet_id identification
0        0x28cc61           test
1        0x29e452          train
2        0x2b3819          train
3        0x2db41f           test
4        0x2a2acc          train
...           ...            ...
1867530  0x227e25          train
1867531  0x293813          train
1867532  0x1e1a7e          train
1867533  0x2156a5          train
1867534  0x2bb9d2          train

[1867535 rows x 2 columns]

In [5]:
#Creating Emotion DF
emotion = "Twitter/emotion.csv"
twitemo_df = pd.read_csv(emotion)
twitemo_df

tweet_id       emotion
0        0x3140b1       sadness
1        0x368b73       disgust
2        0x296183  anticipation
3        0x2bd6e1           joy
4        0x2ee1dd  anticipation
...           ...           ...
1455558  0x38dba0           joy
1455559  0x300ea2           joy
1455560  0x360b99          fear
1455561  0x22eecf           joy
1455562  0x2fb282  anticipation

[1455563 rows x 2 columns]

In [6]:
#Combining DataFrames
dfs = [twitterDM_df,twitid_df,twitemo_df]
twitcom_df = pd.concat([x.set_index('tweet_id') for x in dfs], axis=1).reset_index()
twitcom_df

tweet_id                         hashtags  \
0        0x376b20                       [Snapchat]   
1        0x2d5350    [freepress, TrumpLegacy, CNN]   
2        0x28b412                     [bibleverse]   
3        0x1cd5b0                               []   
4        0x2de201                               []   
...           ...                              ...   
1867530  0x316b80  [mixedfeeling, butimTHATperson]   
1867531  0x29d0cb                               []   
1867532  0x2a6a4f                               []   
1867533  0x24faed                               []   
1867534  0x34be8c                    [Sundayvibes]   

                                                      text identification  \
0        People who post "add me on #Snapchat" must be ...          train   
1        @brianklaas As we see, Trump is dangerous to #...          train   
2        Confident of your obedience, I write to you, k...           test   
3                      Now ISSA is stalking Tasha 😂😂😂 <LH>          train   
4        "Trust is not the same as faith. A friend is s...           test   
...                                                    ...            ...   
1867530  When you buy the last 2 tickets remaining for ...           test   
1867531  I swear all this hard work gone pay off one da...           test   
1867532  @Parcel2Go no card left when I wasn't in so I ...           test   
1867533  Ah, corporate life, where you can date <LH> us...          train   
1867534             Blessed to be living #Sundayvibes <LH>          train   

              emotion  
0        anticipation  
1             sadness  
2                 NaN  
3                fear  
4                 NaN  
...               ...  
1867530           NaN  
1867531           NaN  
1867532           NaN  
1867533           joy  
1867534           joy  

[1867535 rows x 5 columns]

In [7]:
#Extracting the Training and testing Data
twitTrain_df = twitcom_df.loc[twitcom_df['identification'] == 'train']
twitTest_df = twitcom_df.loc[twitcom_df['identification'] == 'test']


### Indexes had to be reset for other uses after everything was separated.

In [8]:
#Reseting and Testing Train Dataframe
twitTrain_df.reset_index(drop=True, inplace=True)
twitTrain_df

tweet_id                       hashtags  \
0        0x376b20                     [Snapchat]   
1        0x2d5350  [freepress, TrumpLegacy, CNN]   
2        0x1cd5b0                             []   
3        0x1d755c      [authentic, LaughOutLoud]   
4        0x2c91a8                             []   
...           ...                            ...   
1455558  0x321566              [NoWonder, Happy]   
1455559  0x38959e                             []   
1455560  0x2cbca6                     [blessyou]   
1455561  0x24faed                             []   
1455562  0x34be8c                  [Sundayvibes]   

                                                      text identification  \
0        People who post "add me on #Snapchat" must be ...          train   
1        @brianklaas As we see, Trump is dangerous to #...          train   
2                      Now ISSA is stalking Tasha 😂😂😂 <LH>          train   
3        @RISKshow @TheKevinAllison Thx for the BEST TI...          train   
4             Still waiting on those supplies Liscus. <LH>          train   
...                                                    ...            ...   
1455558  I'm SO HAPPY!!! #NoWonder the name of this sho...          train   
1455559  In every circumtance I'd like to be thankful t...          train   
1455560  there's currently two girls walking around the...          train   
1455561  Ah, corporate life, where you can date <LH> us...          train   
1455562             Blessed to be living #Sundayvibes <LH>          train   

              emotion  
0        anticipation  
1             sadness  
2                fear  
3                 joy  
4        anticipation  
...               ...  
1455558           joy  
1455559           joy  
1455560           joy  
1455561           joy  
1455562           joy  

[1455563 rows x 5 columns]

In [9]:
#Reseting and Testing Test Dataframe
twitTest_df.reset_index(drop=True, inplace=True)
twitTest_df

tweet_id                           hashtags  \
0       0x28b412                       [bibleverse]   
1       0x2de201                                 []   
2       0x218443  [materialism, money, possessions]   
3       0x2939d5               [GodsPlan, GodsWork]   
4       0x26289a                                 []   
...          ...                                ...   
411967  0x2913b4                                 []   
411968  0x2a980e                                 []   
411969  0x316b80    [mixedfeeling, butimTHATperson]   
411970  0x29d0cb                                 []   
411971  0x2a6a4f                                 []   

                                                     text identification  \
0       Confident of your obedience, I write to you, k...           test   
1       "Trust is not the same as faith. A friend is s...           test   
2       When do you have enough ? When are you satisfi...           test   
3       God woke you up, now chase the day #GodsPlan #...           test   
4       In these tough times, who do YOU turn to as yo...           test   
...                                                   ...            ...   
411967  "For this is the message that ye heard from th...           test   
411968  "There is a lad here, which hath five barley l...           test   
411969  When you buy the last 2 tickets remaining for ...           test   
411970  I swear all this hard work gone pay off one da...           test   
411971  @Parcel2Go no card left when I wasn't in so I ...           test   

       emotion  
0          NaN  
1          NaN  
2          NaN  
3          NaN  
4          NaN  
...        ...  
411967     NaN  
411968     NaN  
411969     NaN  
411970     NaN  
411971     NaN  

[411972 rows x 5 columns]

In [10]:
tweets_train = twitTrain_df[['text','emotion']]
tweets_test = twitTest_df[['text', 'emotion']]

In [11]:
tweets_train.text[50],tweets_train.emotion[50],

('@sydbell1 @strut4cancer £50 donates to Macmillan       👍😎👌 <LH> !!', 'joy')

# 3. Preprocessing
Originally, no preprocessing was done. This was becuase I thought that even without it I would still be able to get a 0.7 or above accuracy. An assumption that proved to be true. With that being said, for some reason or another, I decided to add preprocessing of the data to see how far I could take this. The preprocessing step was mostly to remove usernames, one letter words, unnecessary stopwords, spaces, the lower of text for uniformity and word lemmetimization. I also wanted to do more but I think I did enough. (I got tired). Also tested whether or not removing stop words performed better or worse. You can tell which one I chose based on int's inclusion/exclusion in th immediate following data. Plan to utilize emoji detection as well.

In [12]:
################################# PREVIOUS VERSION####################################
# import nltk
# from nltk.corpus import stopwords
# from nltk.stem import WordNetLemmatizer
# from nltk.tokenize import word_tokenize
# nltk.download('wordnet')
# nltk.download('omw-1.4')
# import re
# import emoji 

# def preprocess_text(Tweet):
#         #Removes Username
#         Tweet = re.sub('@[^\s]+','',Tweet)
        
#         #Demojize Emojis
#         Tweet = emoji.demojize(Tweet)
#         Tweet = Tweet.replace(":"," ")
#         Tweet = ' '.join(Tweet.split())
        
#         #Removes LH
#         Tweet = re.sub('<[^\s]+','',Tweet)
        
#         #Removes Hashtage
#         Tweet = re.sub('#','',Tweet)
        
#         # Remove puntuations and numbers
#         Tweet = re.sub('[^a-zA-Z]', ' ', Tweet)
               
#         # Remove single characters
#         Tweet = re.sub(r"\s+[a-zA-Z]\s+", ' ', Tweet)

#         # remove multiple spaces
#         Tweet = re.sub(r'\s+', ' ', Tweet)
#         Tweet = Tweet.lower()
        
#         # Convert Text sentence to Tokens
#         Tweet = word_tokenize(Tweet)
       
#         #Remove unncecessay stopwords #####Disabled#####
#         stop_words = stopwords.words('english')
#         filtered_text = []
#         for t in Tweet:
#           if t not in stop_words:
#             filtered_text.append(t)

#         # Word lemmatization
#         wordnet_lemmatizer = WordNetLemmatizer()
#         processed_text1 = []
#         for t in filtered_text:
#             word1 = wordnet_lemmatizer.lemmatize(t, pos="n")
#             word2 = wordnet_lemmatizer.lemmatize(word1, pos="v")
#             word3 = wordnet_lemmatizer.lemmatize(word2, pos=("a"))
#             processed_text1.append(word3)

#         result = ""
#         for word in processed_text1:
#             result = result + word + " "
#         result = result.rstrip() 
        
#         return result

# 3.1 Optimized Pre-Processing
After giving up on Pre-Processing aside from Keras, I decide to try it again. The Following method was ahieved through an accidental eureka moment when looking for other Tokenizers and Find Twitter Tokenizer that meshed well with the rest of the code as well as the Keras Module. The follwing model performs better than the previous one by doing everything the previous one did but with inclusion of the emojis themselves. While it may have some very rare symbols issues.
Update:
The issue with the symbol issue that plaugued and significantly reduced to capabilities of the model has been resolved in the new optimized model. This preprocessing converts emojis to connected words, remove username before twittertokenization and hashtag symbols, so that anyword that would have been negatively impacted such as stop words as well has been made to be taken care of. Stop words also removed as well.

In [13]:
################################# UPDATED VERSION WITH BETTER RESULTS###########################

import re
from nltk.tokenize import TweetTokenizer
from cleantext import clean
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import emoji

def preprocess_text(Tweet):
    #Removes LH
    Tweet = re.sub('<[^\s]+','',Tweet)
    
    #Removes Hashtage
    Tweet = re.sub('#','',Tweet)
    
    #Removes Numbers
    Tweet =re.sub(r'[0-9]+', '', Tweet)
        
    #Removes Currency and Emojis
    Tweet = clean(Tweet, no_currency_symbols=True, replace_with_currency_symbol="", no_emoji=True)
    
    #Convets emoji to text
    #Tweet = emoji.demojize(Tweet)
    #Tweet = re.sub('_','',Tweet)
    
    #Remove @Username
    Tweet = re.sub('@[^\s]+','',Tweet)
    
    #Remove Punctation
    Tweet = re.sub(r'[^\w\s]',' ',Tweet)
   

    #Tokenizer Using Tweet Tokenizer
    tknzr = TweetTokenizer(strip_handles=True, reduce_len=True, preserve_case=False)
    Tweet_t = tknzr.tokenize(Tweet)
    
    #Stop Words Removal
    stop_words = stopwords.words('english')
    filtered_text = []
    for t in Tweet_t:
        if t not in stop_words:
            filtered_text.append(t)
            
    # Word lemmatization
    wordnet_lemmatizer = WordNetLemmatizer()
    processed_text1 = []
    for t in filtered_text:
        word1 = wordnet_lemmatizer.lemmatize(t, pos="n")
        word2 = wordnet_lemmatizer.lemmatize(word1, pos="v")
        word3 = wordnet_lemmatizer.lemmatize(word2, pos=("a"))
        processed_text1.append(word3)
    
    result = ""
    for word in processed_text1:
        result = result + word + " "
    result = result.rstrip() 
    return result

In [14]:
train_clean  = []
for c in tweets_train.text:
    train_clean.append(preprocess_text(c))

In [15]:
test_clean  = []
for c in tweets_test.text:
    test_clean.append(preprocess_text(c))

In [16]:
train_clean[::10]

['people post add snapchat must dehydrate cuz man',
 'know research butterfly predictive text autocorrects butterfly justgradstudentthings ecology',
 'never half man pat tillman',
 'follow next hour iphonex windians clevelandindians mothermovie detroithomecoming retweeet bignews',
 'way truth life',
 'donate macmillan',
 'want frank ocean phone cover',
 'anyone want talk anything hit suicide',
 'stop progressive change make whilst battle doubt grind make settle trust jesus',
 'thing world powerful positive push smile world optimism richard devos givehope givetariro',
 'hi suzyq like sibling',
 'want right',
 'fdg',
 'mia',
 'follower bot faketrumpfan sad',
 'okay do need vent twitter',
 'thank follow glad connect wish best life',
 'anything scary restaraunt fountain drink dispenser cough middle fill',
 'always take place want',
 'people never get smh alive well livetheglory',
 'well something right',
 'lol hahah one',
 'thank friyay',
 'ahhh feel free happy queer today good know happen

In [17]:
tweets_train['clean_text'] = train_clean
tweets_test['clean_text'] = test_clean

C:\Users\galve\AppData\Local\Temp\ipykernel_4328\3954056427.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets_train['clean_text'] = train_clean
C:\Users\galve\AppData\Local\Temp\ipykernel_4328\3954056427.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets_test['clean_text'] = test_clean


In [18]:
tweets_test.clean_text

0         confident obedience write know even ask philem...
1         trust faith friend someone trust put faith any...
2         enough satisfy goal really money materialism m...
3                      god wake chase day godsplan godswork
4                               tough time turn symbol hope
                                ...                        
411967      message ye hear begin love one another john kjv
411968    lad hath five barley loaf two small fish among...
411969    buy last ticket remain show sell mixedfeeling ...
411970                       swear hard work go pay one day
411971                           card leave idea get parcel
Name: clean_text, Length: 411972, dtype: object

In [19]:
len_text = []

for text in tweets_train.clean_text:
    len_tweet = len(text.split())
    len_text.append(len_tweet)
    
tweets_train['len_text'] = len_text

C:\Users\galve\AppData\Local\Temp\ipykernel_4328\1782234018.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets_train['len_text'] = len_text


In [20]:
tweets_train.head()

text       emotion  \
0  People who post "add me on #Snapchat" must be ...  anticipation   
1  @brianklaas As we see, Trump is dangerous to #...       sadness   
2                Now ISSA is stalking Tasha 😂😂😂 <LH>          fear   
3  @RISKshow @TheKevinAllison Thx for the BEST TI...           joy   
4       Still waiting on those supplies Liscus. <LH>  anticipation   

                                          clean_text  len_text  
0    people post add snapchat must dehydrate cuz man         8  
1  see trump dangerous freepress around world tru...         8  
2                                   issa stalk tasha         3  
3  thx best time tonight story heartbreakingly au...         9  
4                           still wait supply liscus         4

In [21]:
len_text = []
for text in tweets_test.clean_text:
    len_tweet = len(text.split())
    len_text.append(len_tweet)
tweets_test['len_text'] = len_text

C:\Users\galve\AppData\Local\Temp\ipykernel_4328\1771897654.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets_test['len_text'] = len_text


In [22]:
tweets_test.head()

text emotion  \
0  Confident of your obedience, I write to you, k...     NaN   
1  "Trust is not the same as faith. A friend is s...     NaN   
2  When do you have enough ? When are you satisfi...     NaN   
3  God woke you up, now chase the day #GodsPlan #...     NaN   
4  In these tough times, who do YOU turn to as yo...     NaN   

                                          clean_text  len_text  
0  confident obedience write know even ask philem...         8  
1  trust faith friend someone trust put faith any...        11  
2  enough satisfy goal really money materialism m...         8  
3               god wake chase day godsplan godswork         6  
4                        tough time turn symbol hope         5

In [23]:
#Dropping any tweet that had 0 or 3 or less text after cleaning (Sibject,Verb,Object)
tweets_train = tweets_train[tweets_train['len_text'] > 3]

In [24]:
#Dropping any tweet that had 0 or 3 or less text after cleaning (Sibject,Verb,Object)
tweets_test = tweets_test[tweets_test['len_text'] > 3]

# 3.2 Cleaining and Tokenization Bert

In [25]:
from transformers import BertTokenizerFast
from transformers import TFBertModel

In [26]:
bert_tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')


In [27]:
lens_tokens = []

for i,txt in enumerate(tweets_train['clean_text'].values):
    tokens = bert_tokenizer.encode(txt, max_length=512, truncation=True)
    lens_tokens.append(len(tokens))
tweets_train['len_tokens'] = lens_tokens

In [28]:
tweets_train = tweets_train.sort_values(by='len_tokens', ascending=False)
tweets_train.head(50)


text       emotion  \
476361   #2017年今の与党に投票してあなたの愛を守れますか？ #2017年今の与党に投票してあなた...           joy   
397669   #加藤シゲアキ誕生祭 #加藤シゲアキ20代最後の日 #加藤シゲアキ #加藤シゲアキ30歳 #...           joy   
693853   #Aqours2ndLIVE  #Aqours #小林愛香 #降旗愛 #高槻かなこ #逢田梨...       sadness   
307900   <LH> #스밍 <LH> #워너블 #윤지성 #하성운 #황민현 #옹성우 #김재환 #강...           joy   
1189821  #쿠키런에서_나는_어떤_쿠키 #노츄 #쿠키생일 <LH> JKday #２０２１ #성인...           joy   
1242743  #GENERATIONS <LH> CYCLONE #ライブビューイング  #京成ローザ #...         anger   
428673   #Not_all_karmys_opinion #아미_마지막툽_집중해  #ARMY_FO...           joy   
284989   Related Instagram tags for #Love: #purble #bri...           joy   
178837   #デタカ交換 #ジャニーズJr #Mr.KING #Prince #SixTONES #Lo...           joy   
1391544  #JABHARRYMETSEJAL nly 10dys lft #1 v dnt hv al...       sadness   
780506   Younh And Rich WOO <LH> #종현콘서트_INSPIRED #INSPI...         trust   
897680   new song (only one you need) ? <LH> #종현콘서트_INS...         trust   
660192   featuring shawol 😘 <LH> #종현콘서트_INSPIRED #INSPI...         trust   
131988   featuring shawol 😍😘  <LH> #종현콘서트_INSPIRED #INS...         trust   
1076889  #Dear #Tweeters_can_any_one_follow_me #idosen'...  anticipation   
1197304  @zukkimonn #インスタ #難しい #やろうと思えば #きっとやれる #きっとできる...  anticipation   
1239936  Young And Rich WOO <LH> #종현콘서트_INSPIRED #INSPI...         trust   
152667   warm winter (our season) <LH> #종현콘서트_INSPIRED ...         trust   
868329   171110 Wanna One | 'Beautiful' MV POSTER <LH> ...           joy   
933991   Crazy (Guilty Pleasure) <LH> #종현콘서트_INSPIRED #...         trust   
575572   Wrt, Japanese on Twitter. It's a little viral ...           joy   
523922   new song? <LH> #종현콘서트_INSPIRED #INSPIRED_박수칠준비...         trust   
915789   Beautiful Tonight <LH> #종현콘서트_INSPIRED #INSPIR...         trust   
10223    Happy Birthday <LH> #종현콘서트_INSPIRED #INSPIRED_...         trust   
994386   LoveBelt <LH> #종현콘서트_INSPIRED #INSPIRED_박수칠준비되...         trust   
1325707  I'm so tired 😔#NewZealandGPG #정세운데뷔축하해 #taeyeo...           joy   
449911   new song again? <LH> #종현콘서트_INSPIRED #INSPIRED...         trust   
291185   #こぼれる声を #聴かせて #あなたとハネムーン #祝福してる #世界中がエキストラ #永遠...           joy   
625369   Love Belt <LH> #종현콘서트_INSPIRED #INSPIRED_박수칠준비...         trust   
151620   Love Is So Nice <LH> #종현콘서트_INSPIRED #INSPIRED...         trust   
861248   END OF A DAY <LH> #종현콘서트_INSPIRED #INSPIRED_박수...         trust   
696209   Like You <LH> #종현콘서트_INSPIRED #INSPIRED_박수칠준비되...         trust   
873662   Cocktail <LH> #종현콘서트_INSPIRED #INSPIRED_박수칠준비되...         trust   
32620    Inspiration <LH> #종현콘서트_INSPIRED #INSPIRED_박수칠...         trust   
512496   NEON <LH> #종현콘서트_INSPIRED #INSPIRED_박수칠준비되셨나 #...         trust   
488192   LONELY <LH> #종현콘서트_INSPIRED #INSPIRED_박수칠준비되셨나...         trust   
320504   Moon <LH> #종현콘서트_INSPIRED #INSPIRED_박수칠준비되셨나 #...         trust   
495780   Dress Up <LH> #종현콘서트_INSPIRED #INSPIRED_박수칠준비되...         trust   
85864    Let Me Out <LH> #종현콘서트_INSPIRED #INSPIRED_박수칠준...         trust   
1305260  Orbit <LH> #종현콘서트_INSPIRED #INSPIRED_박수칠준비되셨나 ...         trust   
666926   #GirIs' TaIk #もう一度~one more time~ #0~ZERO #Lov...           joy   
960374   No more sad songs!#하프밀리언 #1YearInBLACKPINKsAre...           joy   
166270   E_mhotep DRKYSELA sprague_paul spnn52n9 articl...           joy   
244865   <LH> #Wishes for all #Womens in #SaudiArabia #...         trust   
795284   SHE IS <LH> #종현콘서트_INSPIRED #INSPIRED_박수칠준비되셨나...         trust   
176670   ONGNIEL IS PERFECT !! <LH> <LH> <LH> <LH> #강다니...           joy   
72308    what happen to wanna one's make up ? <LH> <LH>...           joy   
902000   <LH> #IHaveADream #IStillHaveADream #ImplicitB...         trust   
1311936  #Blind=#DontBelieve$Dies #Deaf=#DontBelieveGod...       disgust   
244504   #NewShairy #Best_Shairy #Dukhi_Shairy #Nice_Sh...           joy   

                                                clean_text  len_text  \
476361   nian jin noyu dang nitou piao shiteanatanoai w...        43   
397669   jia teng shigeakidan she

In [29]:
#Removal of any excessive tokenization tweet
tweets_train = tweets_train[tweets_train['len_tokens'] <= 50]
tweets_train.head()

text emotion  \
523922   new song? <LH> #종현콘서트_INSPIRED #INSPIRED_박수칠준비...   trust   
915789   Beautiful Tonight <LH> #종현콘서트_INSPIRED #INSPIR...   trust   
10223    Happy Birthday <LH> #종현콘서트_INSPIRED #INSPIRED_...   trust   
994386   LoveBelt <LH> #종현콘서트_INSPIRED #INSPIRED_박수칠준비되...   trust   
1325707  I'm so tired 😔#NewZealandGPG #정세운데뷔축하해 #taeyeo...     joy   

                                                clean_text  len_text  \
523922   new song jonghyeonkonseoteu_inspired inspired_...         9   
915789   beautiful tonight jonghyeonkonseoteu_inspired ...         9   
10223    happy birthday jonghyeonkonseoteu_inspired ins...         9   
994386   lovebelt jonghyeonkonseoteu_inspired inspired_...         8   
1325707  tire newzealandgpg jeongseundebwicughahae taey...        12   

         len_tokens  
523922           50  
915789           50  
10223            50  
994386           50  
1325707          50

In [30]:
#Reshuffle and Reset index
tweets_train = tweets_train.sample(frac=1).reset_index(drop=True)

In [31]:
lens_tokens = []

for i,txt in enumerate(tweets_test['clean_text'].values):
    tokens = bert_tokenizer.encode(txt, max_length=512, truncation=True)
    lens_tokens.append(len(tokens))
tweets_test['len_tokens'] = lens_tokens

In [32]:
tweets_test = tweets_test.sort_values(by='len_tokens', ascending=False)
tweets_test.head(20)

text emotion  \
81415   #صباحات_النصر  pin5912DC0D #سكس_كتابي #المدينة...     NaN   
399380  How @parcelforce hv faild ths wk: Tues drvr cl...     NaN   
356504  @MSNBC Hee haz kno respekt cuz hee didnnt nock...     NaN   
148629  2 cmpltly #TrustInALLAH z 2 b lyk a child who ...     NaN   
314812  @OfficeOfRG hw cm u cl ppl of GJ Gundas; let m...     NaN   
186270  At da height of 13K fts, VIPs of eithr cydr wr...     NaN   
357690  Colossians1:27 2whomGODwouldMAKEknownWHAT ISth...     NaN   
407805  #SiezePaddocksAssets  #SiezePaddocksAssets  #S...     NaN   
367835  #DontEndKRPKAB #KRPKABians Were Busy Seeing #I...     NaN   
41912   MoreFAKEnews is generated & pushed by FOX NEWs...     NaN   
265153  Gurmt ramrahim sentncd to 10yrs in jail fr rap...     NaN   
357266  #GodAndPills #TestTheSpirit #IAlwaysKnewThereW...     NaN   
410022  ihasjohn_bob ELMO! NOOOO! Haha thanks John Bob...     NaN   
193963  Wsh u all Hppy Anant Chaturdashi.Pray To God F...     NaN   
182462  #MONSTA_X  170722  Beautiful jap ver MV teaser...     NaN   
166434  K3G : Multistarrer OSO : Multistarrer CE : Bco...     NaN   
286018  Mets lineup vs. DBacks:   Nimmo CF Cabrera 2B ...     NaN   
309636  I fucking love crepes ❤️❤️❤️❤️ #crepes#itslit ...     NaN   
182416  @capt_amarinder Sir, Punjab ch #electricity de...     NaN   
11062   Haha pukhtoon ?? Pukhtoon are nt lyk ths bibi ...     NaN   

                                               clean_text  len_text  \
81415   sbht_lnsr pindcd sks_ktby lmdyn k mhnh nyk zb ...        21   
399380  hv faild th wk tues drvr cldnt use drbll ystrd...        21   
356504  hee haz kno respekt cuz hee didnnt nock owt mi...        21   
148629  cmpltly trustinallah z b lyk child knz dply dt...        24   
314812  hw cm u cl ppl gj gundas let ask u hw u go gj ...        26   
186270  da height k ft vip eithr cydr wre gv spcial pe...        21   
357690  colossian whomgodwouldmakeknownwhat istheriche...         9   
407805  siezepaddocksassets siezepaddocksassets siezep...         6   
367835  dontendkrpkab krpkabians busy see ishqbaaz bt ...        14   
41912   morefakenews generate push fox news d_trump tr...        10   
265153  gurmt ramrahim sentncd yr jail fr rape im real...        17   
357266  godandpills testthespirit ialwaysknewtherewasa...        10   
410022  ihasjohn_bob elmo nooo haha thank john bob hum...        16   
193963  wsh u hppy anant chaturdashi pray god fr ur pr...        23   
182462  monsta_x beautiful jap v mv teaser monseutaegs...        16   
166434  kg multistarrer oso multistarrer ce bcoz dp ro...        20   
286018  mets lineup v dbacks nimmo cf cabrera b confor...        22   
309636  fuck love crepe crepesitslit crepesarelife abu...         8   
182416  sir punjab ch electricity de problm da v kuch ...        18   
11062   haha pukhtoon pukhtoon nt lyk th bibi u prtctv...        17   

        len_tokens  
81415           58  
399380          47  
356504          45  
148629          45  
314812          44  
186270          43  
357690          43  
407805          43  
367835          42  
41912           41  
265153          41  
357266          40  
410022          40  
193963          40  
182462          40  
166434          39  
286018          39  
309636          39  
182416          39  
11062           39

In [33]:
#Removal of any excessive tokenization tweet
tweets_test = tweets_test.iloc[12:]
tweets_test.head()

text emotion  \
410022  ihasjohn_bob ELMO! NOOOO! Haha thanks John Bob...     NaN   
193963  Wsh u all Hppy Anant Chaturdashi.Pray To God F...     NaN   
182462  #MONSTA_X  170722  Beautiful jap ver MV teaser...     NaN   
166434  K3G : Multistarrer OSO : Multistarrer CE : Bco...     NaN   
286018  Mets lineup vs. DBacks:   Nimmo CF Cabrera 2B ...     NaN   

                                               clean_text  len_text  \
410022  ihasjohn_bob elmo nooo haha thank john bob hum...        16   
193963  wsh u hppy anant chaturdashi pray god fr ur pr...        23   
182462  monsta_x beautiful jap v mv teaser monseutaegs...        16   
166434  kg multistarrer oso multistarrer ce bcoz dp ro...        20   
286018  mets lineup v dbacks nimmo cf cabrera b confor...        22   

        len_tokens  
410022          40  
193963          40  
182462          40  
166434          39  
286018          39

In [34]:
#Reset Index
tweets_test = tweets_test.sample(frac=1).reset_index(drop=True)

# 3.3 Label Creation

In [35]:
tweets_train['emotion'].value_counts()


joy             414631
anticipation    214284
trust           169876
sadness         156859
disgust         118381
fear             45451
surprise         38980
anger            34836
Name: emotion, dtype: int64

In [36]:
tweets_train['emotion'] = tweets_train['emotion'].map({'joy':0, 'anticipation':1, 'trust':2, 'sadness':3, 'disgust':4, 'fear':5, 'surprise':6, 'anger':7})

In [37]:
tweets_train['emotion'].value_counts()

0    414631
1    214284
2    169876
3    156859
4    118381
5     45451
6     38980
7     34836
Name: emotion, dtype: int64

In [38]:
classes = ['joy', 'anticipation', 'trust', 'sadness', 'disgust', 'fear', 'surprise', 'anger']


In [39]:
classes_to_index = dict((c, i) for i, c in enumerate(classes))
index_to_classes = dict((v, k) for k, v in classes_to_index.items())

In [40]:
classes_to_index

{'joy': 0,
 'anticipation': 1,
 'trust': 2,
 'sadness': 3,
 'disgust': 4,
 'fear': 5,
 'surprise': 6,
 'anger': 7}

In [41]:
index_to_classes

{0: 'joy',
 1: 'anticipation',
 2: 'trust',
 3: 'sadness',
 4: 'disgust',
 5: 'fear',
 6: 'surprise',
 7: 'anger'}

In [42]:
#Function used to convert string values to numeric values to be used in training along with the padded data.
names_to_ids = lambda labels: np.array([classes_to_index.get(x) for x in labels])
ids_to_names = lambda labels: np.array([index_to_classes.get(x) for x in labels])

# 3.4 Class Balancing and Data Spliting

In [43]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler()

train_x, train_y = ros.fit_resample(np.array(tweets_train['clean_text']).reshape(-1, 1), np.array(tweets_train['emotion']).reshape(-1, 1));
train_pad = pd.DataFrame(list(zip([x[0] for x in train_x], train_y)), columns = ['clean_text', 'emotion']);

In [44]:
train_pad['emotion'].value_counts()

4    414631
5    414631
6    414631
3    414631
1    414631
0    414631
2    414631
7    414631
Name: emotion, dtype: int64

In [45]:
train_pad.head()

clean_text  emotion
0         would continue buy season ticket ref chump        4
1  china allow business country america new chine...        5
2  distraction distract trumprussia trumpcrimefam...        6
3   student unprepared disengage exhaust teacherlife        3
4  always never let anyone destroy yourdreams wou...        1

In [46]:
#Creating variables to use in train_test_split
X = train_pad['clean_text'].values
Y = train_pad['emotion'].values

In [47]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


In [48]:
print(X_train[50])
print(y_train[50])

side note shout w presence audience kudos rock bizgalz hesm
0


In [49]:
#
tweets_pred_test = tweets_test['clean_text']

In [50]:
print(tweets_pred_test[50])


god name use strong head tail misuse weak break stupid use name


In [51]:
#During the process of creating and deleting my many models, one hot encoding provide more accurate results
from sklearn import preprocessing

ohe = preprocessing.OneHotEncoder()
y_train = ohe.fit_transform(np.array(y_train, dtype=object).reshape(-1, 1)).toarray()
y_valid = ohe.fit_transform(np.array(y_test, dtype=object).reshape(-1, 1)).toarray()


# 4 Bert Modeling Process

## 4.2 Encoding_Plus

In [52]:
import tensorflow as tf


In [53]:
import torch

In [54]:
len_max=128

In [63]:
#Function for Bert.Encode_Plus
def tokenize(data,max_len=len_max) :
    input_ids = []
    attention_masks = []
    for i in range(len(data)):
        encoded = bert_tokenizer.encode_plus(
            data[i],
            add_special_tokens=True,
            max_length=len_max,
            padding='max_length',
            #return_tensors='tf',
            return_attention_mask=True
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
    
    # Convert lists to tensors
    #input_ids = tf.convert_to_tensor(input_ids)
    #attention_masks = tf.convert_to_tensor(attention_masks)

    return np.array(input_ids, dtype =np.int32),np.array(attention_masks, dtype=np.int32)

In [64]:
test = ['big sister always see eye eye come kid always look raisinghandsmedium darkskintone faceblowingakiss revolvinghearts thankful','This is a Test']

In [65]:
idTest, masktTest = tokenize(test, len_max)

In [66]:
idTest[1]

array([ 101, 2023, 2003, 1037, 3231,  102,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0])

In [67]:
inTestTensor =tf.convert_to_tensor(idTest)

In [68]:
inTestTensor

<tf.Tensor: shape=(2, 128), dtype=int32, numpy=
array([[  101,  2502,  2905,  2467,  2156,  3239,  3239,  2272,  4845,
         2467,  2298,  6274, 11774,  6491,  2098,  5007,  2601, 29334,
         5524,  2227, 16558, 14138,  8978,  4757, 24135, 22375,  2015,
        18836,   102,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0, 

In [69]:
train_inid, train_attmask = tokenize(X_train, len_max)

In [75]:
val_inid, val_attmask = tokenize(X_test, len_max)

In [76]:
test_inid, test_attmask = tokenize(tweets_pred_test, len_max)

In [ ]:
# train_inid, train_attmask = tokenize(X_train, len_max)
# val_inid, val_attmask = tokenize(X_test, len_max)
# test_inid, test_attmask = tokenize(tweets_pred_test, len_max)

In [ ]:
# train_inid_tf = tf.convert_to_tensor(train_inid)
# train_attmask_tf = tf.convert_to_tensor(train_attmask)
# val_inid_tf = tf.convert_to_tensor(val_inid)
# val_attmask_tf = tf.convert_to_tensor(val_attmask)
# test_inid_tf = tf.convert_to_tensor(test_inid)
# test_attmask_tf = tf.convert_to_tensor(test_attmask)

## 4.2 Model Creation

In [70]:
bert_model = TFBertModel.from_pretrained('bert-base-uncased')


Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [71]:
#metrics
from sklearn.metrics import accuracy_score, f1_score

In [72]:
import tensorflow as tf
opt = tf.keras.optimizers.Adam(learning_rate=1e-5, decay=1e-7)
loss = tf.keras.losses.CategoricalCrossentropy()
accuracy = tf.keras.metrics.CategoricalAccuracy()


input_ids = tf.keras.Input(shape=(len_max,),dtype='int32')
attention_masks = tf.keras.Input(shape=(len_max,),dtype='int32')
embeddings = bert_model([input_ids,attention_masks])[1]
output = tf.keras.layers.Dense(8, activation="softmax")(embeddings)

model = tf.keras.models.Model(inputs = [input_ids,attention_masks], outputs = output)

model.compile(opt, loss=loss, metrics=accuracy)
    
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 128)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  109482240   ['input_1[0][0]',                
                                thPoolingAndCrossAt               'input_2[0][0]']                
                                tentions(last_hidde                                               
                                n_state=(None, 128,                                           

In [73]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
callbacks_list = [tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=1,)]

In [79]:
history_bert = model.fit([train_inid, train_attmask], y_train, 
                         validation_data=([val_inid,val_attmask], y_valid), 
                         epochs=10,
                         batch_size=8,
                         callbacks = callbacks_list
                         )

Epoch 1/10
  1205/331705 [..............................] - ETA: 31:29:46 - loss: 1.8627 - categorical_accuracy: 0.2783

KeyboardInterrupt: 

# 4 Tokenization
As mentioned before, certain decision were decided due to the speed of how things could be. Here is where that started, with usage of Tensorflow, Keras, and trying to make code that make things faster but instead made things longer. I decided to go with the Keras Tokenization as I was able to set a limit of words to be used, as well as it being very compatible with padding. This solution performed well as it was able to still provided a relatively decent accuracy predicition overall.

### Labels were set and functions were created that could easily convert the emotions to a value to be used in the training of the model and convert those values back to string.


# MODEL CREATION
#### Genuinely just relied on Google for the formatting and trial and error.
Model was created based on RNN and utilized the Keras Layers to aid with long-form content by using LSTM to mitigate the initial proplems with RNN which is it's potentially to lean towards improper weight gradients when it comes to large data looping too much and messing with the weight values.
With all that being said, even without pre-processing it still performed relatively well. The Original Training of the Model was done at batches of 64, to save time and Epochs of 25. The 64, which is larger than the usual 32, may have been faster but could have impacted accuracy as well. As such, in the second round with the preprocessed data, it was changed to 32 and 10. The model utilize Keras embedding and layers to create an RNN that checks for meaning withing the word and related words of the twitter. The LSTM allows for the RNN to avoid looping too much and messing with the ability to validate data properly. The Dense is amount to amount of emotions and utilize softmax for the predicted results.

# Model Training
No validation data sure made things interesting.
Original Training = 18/25 Epoch for Batches of 64.
Current = 10 Epoch for Batches of 32.
There was an issue whereby the predit function did not produce the desired results and I thought the model was wrong. However, it turned out I was using an outdated TensorFlow model function and I have learned the importance of reading a Module information and formatting. The working model produce a single value which is then converted from index_to_ids from the previously created function. The most optimal set of intructions and parameter are what we have now in this document. 

The various versions were tested on a 1 Epoch basis, and the current model, the No Pre-Processing Model consistently placed the best. In order of Performance, the other ones were Model With Stop Words and Demojize Include and the Last was the Pre-Processing Model that got rid of most things.

That being said, while this first model was indeed the best, by working with the other models I was able narrow down the num_words, length, and epochs for more or less the best performance I could do.


In [ ]:
#Predicting
pred = np.argmax(model.predict([test_inid,test_attmask]), axis=-1)


In [ ]:
pred[50]

In [ ]:
pred.shape, tweets_test_pad.shape

### Converting Prediction index to name and comparing test results.

In [ ]:
pred_result = ids_to_names(pred)
pred_result[:5]

In [ ]:
print(tweets_test[50])
print(pred_result[50])

In [ ]:
my_submission = pd.DataFrame({'id': twitTest_df.tweet_id, 'emotion': pred_result})
my_submission.to_csv('submission.csv', index=False)

### Optimized Model outperforms other Pre-Processing Models but not the Main Model